In [ ]:
# initial setup
%run "../../../common/0_notebooks_base_setup.py"


---

<img src='../../../common/logo_DH.png' align='left' width=35%/>


## Introducción

En este checkpoint vamos a realizar un modelo de clasificación basado en árboles de decisión para predecir si una pelicula va a obtener un premio oscar o no. Veremos como graficar los árboles de decisión obtenidos y como varía la precisión luego de limitar la profundidad del mismo.


## Dataset

En esta clase usaremos un dataset con info de películas ("Movie_classification.csv").  
Este dataset esta conformado por los siguientes features:  

 *   **Marketing expense:**    (float64)    Gasto total en Marketing      
 *   **Production expense:**   (float64)    Gasto total de Producción
 *   **Multiplex coverage:**   (float64)    Cobertura promedio de Multiplex
 *   **Budget:**               (float64)    Presupuesto
 *   **Movie_length:**         (float64)    Duración de la película
 *   **Lead_ Actor_Rating:**   (float64)    Puntaje sobre el actor principal
 *   **Lead_Actress_rating:**  (float64)    Puntaje sobre la actriz principal
 *   **Director_rating:**      (float64)    Puntaje sobre el Director
 *   **Producer_rating:**      (float64)    Puntaje sobre el Productor
 *   **Critic_rating:**        (float64)    Puntaje que le puso la crítica
 *   **Trailer_views:**        (int64)      Cantidad de vistas del Trailer
 *   **3D_available:**         (object)     Si esta disponible en 3D (Yes/No)
 *   **Time_taken:**           (float64)    Duración de la película
 *   **Twitter_hastags:**      (float64)    Cantidad de menciones en twitter
 *   **Genre:**                (object)     Genero de la película
 *   **Avg_age_actors:**       (int64)      Edad promedio de los actores
 *   **Num_multiplex:**        (int64)      Cantidad de Multiplex
 *   **Collection:**           (int64)      Recaudación
 *   **Start_Tech_Oscar:**     (int64)      Si recibió un oscar o no.
 
 


## Imports

In [ ]:
import numpy as np
import pandas as pd
#import seaborn as sns
#import matplotlib.pyplot as plt

## Ejercicio 1 - Importar datos

1) Leamos los datos de "Movie_classification.csv" y lo guardamos en un dataframe de pandas.  
2) Veamos cuántos registros hay en cada DataFrame y de qué tipos son los datos de cada columna.   
3) Veamos los primeros registros de cada DataFrame para verificar que los datos fueron importados correctamente.

In [ ]:
df = pd.read_csv("../Data/Movie_classification.csv", header=0)

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.head()

## Ejercicio 2 - Imputación de valores faltantes

Veamos si existen valores faltantes y en tal caso imputemos los mismos.

In [ ]:
#vemos la cantidad de valores que tiene cada columna
df.info()

In [ ]:
# observamos que time_taken es la unica columna que no tiene 506 observaciónes, 
# por ende vamos a imputar los valores faltantes utilizando la media.
df['Time_taken'].mean()

In [ ]:
df['Time_taken'].fillna(value = df['Time_taken'].mean(), inplace = True)

In [ ]:
df.info()

## Ejercicio 3 - Generación de Variables Dummies.

Veamos si existen variables categóricas y en tal caso generar variables dummies para dichas columnas.

In [ ]:
df.dtypes.loc[df.dtypes=="object"]  

In [ ]:
df[['3D_available','Genre']].head()

In [ ]:
df.Genre.unique()

In [ ]:
df["3D_available"].unique()

In [ ]:
df = pd.get_dummies(df,columns = ["3D_available","Genre"],drop_first = True)

In [ ]:
df.head()

## Ejercicio 4 - Features, Target

Construyamos una matriz de features (X) y el vector target (Y) para predecir `Start_Tech_Oscar` en el dataset de datos completos

¿Qué valores toma la variable `Start_Tech_Oscar` en el dataset?


In [ ]:
df.columns!="Start_Tech_Oscar"

In [ ]:
X = df.loc[:,df.columns!="Start_Tech_Oscar"]
type(X)

In [ ]:
X.head()

In [ ]:
X.shape

In [ ]:
y = df["Start_Tech_Oscar"]
type(y)

In [ ]:
y.head()

In [ ]:
y.shape

## Ejercicio 5 - Train Test Split

Constuyamos los conjuntos de train y test, asignando el 70% de los registros a train y el 30% a test

https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.3,random_state=42)

In [ ]:
X_train.head()

In [ ]:
X_train.shape

In [ ]:
X_test.shape

## Ejercicio 6 - Entrenamos el árbol de clasificación
https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html  

Entrenemos el árbol de clasificación y realizar las predicciones tanto para el dataset de entrenamiento como el de pruebas


In [ ]:
from sklearn import tree
clftree = tree.DecisionTreeClassifier(random_state=40)

In [ ]:
clftree.fit(X_train, y_train)

In [ ]:
y_train_pred = clftree.predict(X_train)
y_test_pred = clftree.predict(X_test)

## Ejercicio 7 - Verificamos la performance del modelo
https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html  
https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html  

Importemos las librerías de `accuracy_score`  y `confusion_matrix` para analizar la performance de nuestro modelo tanto para el dataset de entrenamiento como el de pruebas 


Recordamos que la matriz de confusión tiene la siguiente forma:

<img style="float: center;" src="img/matriz_confusion.png">

<img style="float: center;" src="img/matriz_confusion_2.png">

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix

In [ ]:
confusion_matrix(y_train, y_train_pred)

In [ ]:
confusion_matrix(y_test, y_test_pred)

In [ ]:
accuracy_score(y_test, y_test_pred)

Los árboles de decisión implementados en scikit-learn ofrecen un atributo llamado `feature_importances_`. Es la contribunción a la reducción de las impurezas a lo largo de todo el árbol. 

In [ ]:
pd.DataFrame({'atributo':X.columns, 'importancia':clftree.feature_importances_}).sort_values('importancia', ascending = False)

## Ejercicio 8 - Graficamos el modelo de ML.

Es necesario tener instaladas las librerias de `graphviz` y `pydotplus`

```
> conda install graphviz
> pip install pydotplus
```

https://scikit-learn.org/stable/modules/generated/sklearn.tree.export_graphviz.html  
https://chrisalbon.com/machine_learning/trees_and_forests/visualize_a_decision_tree/


In [ ]:
dot_data = tree.export_graphviz(clftree, out_file=None,feature_names= X_train.columns, filled = True)

In [ ]:
from IPython.display import Image

In [ ]:
import pydotplus

In [ ]:
graph = pydotplus.graph_from_dot_data(dot_data)
Image(graph.create_png())

## Ejercicio 9 - Analizamos como varía la performance al reducir la altura del árbol (tree prunning).

Realizamos tree prunning definiendo los siguientes hiper-parámetros:
 - min_samples_leaf = 20
 - max_depth = 4  
 
Luego graficamos el árbol entrenado y verificamos la performance en el dataset de pruebas.
 

In [ ]:
clftree2 = tree.DecisionTreeClassifier(min_samples_leaf = 20, max_depth=4, random_state=40)
clftree2.fit(X_train, y_train)
dot_data = tree.export_graphviz(clftree2, out_file=None,feature_names= X_train.columns, filled = True)
graph2 = pydotplus.graph_from_dot_data(dot_data)
Image(graph2.create_png())

In [ ]:
accuracy_score(y_test, clftree2.predict(X_test))

In [ ]:
confusion_matrix(y_test, clftree2.predict(X_test))

In [ ]:
pd.DataFrame({'atributo':X.columns, 'importancia':clftree2.feature_importances_}).sort_values('importancia', ascending = False)

## Ejercicio 10 - Optimizamos nuestro árbol de decisión ajustando los hiperparámetros mediante el uso de gridsearch.

https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html  

Optimizamos el modelo, aplicando gridseach con los siguientes hiperparametros:

```python 
params_grid = { "criterion" : ["gini", "entropy"],
                "min_samples_leaf": [5,10,15,20,None], 
                "max_depth" : [1,2,3,4,5,6,8,9,10,11,12,13,14,15,16,17,None],
                "min_samples_split": [2, 3, 4,None]}
``` 



In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
clftree = tree.DecisionTreeClassifier(random_state=40)

In [ ]:
params_grid = { "criterion" : ["gini", "entropy"],
                "min_samples_leaf": [5,10,15,20,None], 
                "max_depth" : [1,2,3,4,5,6,8,9,10,11,12,13,14,None],
                "min_samples_split": [2, 3, 4,None]}

In [ ]:
grid_search = GridSearchCV(clftree, params_grid,
                           n_jobs=-1, scoring='accuracy')

In [ ]:
grid_search.fit(X_train, y_train)

In [ ]:
#Score durante el entrenamiento.
grid_search.best_score_

In [ ]:
cv_clf = grid_search.best_estimator_

In [ ]:
cv_clf

In [ ]:
accuracy_score(y_test, cv_clf.predict(X_test))

In [ ]:
confusion_matrix(y_test, cv_clf.predict(X_test))

In [ ]:
dot_data = tree.export_graphviz(cv_clf, out_file=None,feature_names= X_train.columns, filled = True)
graph2 = pydotplus.graph_from_dot_data(dot_data)
Image(graph2.create_png())

In [ ]:
pd.DataFrame({'atributo':X.columns, 'importancia':cv_clf.feature_importances_}).sort_values('importancia', ascending = False)